In [1]:
import requests

from bs4 import BeautifulSoup

import lxml

import datetime

import pandas as pd
import numpy as np

In [2]:
baseUrl='http://www.yes24.com/24/category/bestseller{data}?'+ \
        'CategoryNumber=001001046012008&sumgb=09&'+ \
        'year={year}&'+ \
        'month={month}&'+ \
        'FetchSize=80&'+ \
        'PageNumber={PageNumber}'

In [3]:
def getPages(year, month, data):
    url = baseUrl.format(year=year, month=month, data=data, PageNumber='1')
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')

    a_list = soup.select('div[id==bestList] div[class==sortLft] p[class=page] a[class=page_href]')
    pages = int(len(a_list)/2)+1

    return pages

In [4]:
dt = datetime.datetime.now()
today = dt.strftime("%Y%m%d")

max_page_number = 50
start_day = '20180201'

dt_index = pd.date_range(start=start_day, end=today)
dt_list = list(set(dt_index.strftime("%Y,%m").tolist()))
dt_list.sort()

In [5]:
def cropPage(req):
    soup = BeautifulSoup(req.text, 'lxml')

    trs = soup.select('table tr')

    nth_line = []
    nth_lines = []

    for tr in trs:
        tds =tr.select('td')
        for idx, td in enumerate(tds):
            if idx in [0,1,3,4,6,7,10]:
                nth_line.append(td.string)
        nth_lines.append(nth_line)
        nth_line=[]

    return nth_lines

In [6]:
def requestMonth(year, month, pageNo, data):    
    url = baseUrl.format(year=year, month=month, data=data, PageNumber=pageNo)
    return requests.get(url)

In [7]:
def yes24bestseller(year, month):
    
    pages = getPages(year, month, data='')
    
    for idx in range(pages):
        page = idx+1
        req = requestMonth(year, month, page, data='Excel')
        if idx == 0:
            perPage = cropPage(req)
        perPage += cropPage(req)[1:]
        
    return perPage

In [8]:
each_month = {}
for one in dt_list:
    year, month = one.split(',')
    
    per_month = yes24bestseller(year, month)
    each_month[str(year)+str(month)]=per_month

In [9]:
print(each_month.keys())
each_month['201805']

dict_keys(['201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008'])


[['순위', 'ISBN', '상품명', '판매가', '저자', '출판사', '관리분류'],
 ['1',
  '9791187824244',
  '히어로즈 저니',
  '13,500원',
  '스티브 비흘링/김지윤/김종윤',
  '아르누보',
  '국내도서-소설/시/희곡'],
 ['2',
  '9788934993766',
  '셰이프 오브 워터',
  '13,500원',
  '대니얼 크라우스/Guillermo Del Toro/김문주',
  '온다',
  '국내도서-소설/시/희곡'],
 ['3',
  '9791187824220',
  '인피니티 스톤의 비밀 1',
  '11,700원',
  '브랜던 T. 스나이더/김세은/김종윤',
  '아르누보',
  '국내도서-소설/시/희곡'],
 ['4',
  '9788925563428',
  '지금, 만나러 갑니다',
  '12,420원',
  '이치카와 다쿠지/양윤옥',
  '알에이치코리아(RHK)',
  '국내도서-소설/시/희곡'],
 ['5',
  '9788990982704',
  '용의자 X의 헌신',
  '15,120원',
  '히가시노 게이고/양억관',
  '재인',
  '국내도서-소설/시/희곡'],
 ['6',
  '9788959526345',
  '월드 오브 워크래프트 연대기 3',
  '26,820원',
  '크리스 멧젠/블리자드 엔터테인먼트/고경훈',
  '제우미디어',
  '국내도서-소설/시/희곡'],
 ['7', '9788954622035', '살인자의 기억법', '9,000원', '김영하', '문학동네', '국내도서-소설/시/희곡'],
 ['8',
  '9788960776654',
  '레디 플레이어 원',
  '14,760원',
  '어니스트 클라인/전정순',
  '에이콘출판사',
  '국내도서-소설/시/희곡'],
 ['9',
  '9788937462344',
  '참을 수 없는 존재의 가벼움',
  '9,900원',
  '밀란 쿤데라/이재룡',
  '민음사',
  '국내도서-소설/시/희곡'],
 [